## Setup: Conexión a AstraDB

In [1]:
# Importar librerías necesarias
import sys
from pathlib import Path
from tabulate import tabulate
import pandas as pd

# Agregar el directorio del proyecto al path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

# Importar cliente de AstraDB
from src.config.astradb_config import get_astradb_client

print("Conectando a AstraDB...")
db = get_astradb_client()
print("✓ Conexión establecida")

# Verificar colecciones disponibles
collections = db.list_collection_names()
print(f"\nColecciones disponibles: {collections}")

Conectando a AstraDB...
✓ Conexión establecida

Colecciones disponibles: ['cost_anomalies', 'genai_tokens_daily', 'org_daily_usage', 'org_service_costs', 'revenue_monthly', 'tickets_critical_daily']


In [2]:
# Helper: Obtener un org_id de ejemplo de los datos
def get_sample_org_id():
    collection = db.get_collection("org_daily_usage")
    sample_doc = collection.find_one({})
    if sample_doc and 'org_id' in sample_doc:
        return sample_doc['org_id']
    return None

# Helper: Obtener rango de fechas disponible
def get_date_range():
    collection = db.get_collection("org_daily_usage")
    cursor = collection.find({}, limit=100)
    dates = sorted([doc.get('usage_date') for doc in cursor if doc.get('usage_date')])
    return dates[0] if dates else None, dates[-1] if dates else None

sample_org = get_sample_org_id()
min_date, max_date = get_date_range()
print(f"Org ID de ejemplo: {sample_org}")
print(f"Rango de fechas disponible: {min_date} a {max_date}")

Org ID de ejemplo: org_0lzjjege
Rango de fechas disponible: 2025-07-04 a 2025-08-30


---
## Query 1: Costos y requests diarios por org y servicio (rango de fechas)

**Requerimiento TP**: *"Costos y requests diarios por org y servicio en un rango de fechas."*

### CQL Equivalent:
```sql
SELECT org_id, usage_date, service, total_cost_usd, total_requests 
FROM org_daily_usage 
WHERE org_id = ? AND usage_date >= ? AND usage_date <= ?
ORDER BY usage_date DESC;
```

In [3]:
# QUERY 1: Costos y requests diarios por org y servicio en un rango de fechas

ORG_ID = sample_org
START_DATE = min_date  # Usar fecha mínima disponible
END_DATE = max_date    # Usar fecha máxima disponible

print("=" * 100)
print("QUERY 1: Costos y requests diarios por org y servicio en un rango de fechas")
print("=" * 100)
print(f"Parámetros: org_id={ORG_ID}, start_date={START_DATE}, end_date={END_DATE}")
print()
print("CQL Equivalent:")
print(f"  SELECT org_id, usage_date, service, total_cost_usd, total_requests")
print(f"  FROM org_daily_usage")
print(f"  WHERE org_id = '{ORG_ID}' AND usage_date >= '{START_DATE}' AND usage_date <= '{END_DATE}'")
print(f"  ORDER BY usage_date DESC;")
print()

# Ejecutar query
collection = db.get_collection("org_daily_usage")
cursor = collection.find(
    filter={
        "org_id": ORG_ID,
        "usage_date": {"$gte": START_DATE, "$lte": END_DATE}
    },
    sort={"usage_date": -1},
    limit=20
)

results_q1 = []
for doc in cursor:
    results_q1.append([
        doc.get('org_id', ''),
        doc.get('usage_date', ''),
        doc.get('service', ''),
        f"${float(doc.get('total_cost_usd', 0)):,.2f}",
        f"{int(doc.get('total_requests', 0)):,}"
    ])

headers = ["Org ID", "Date", "Service", "Cost USD", "Requests"]
print(tabulate(results_q1, headers=headers, tablefmt="grid"))
print(f"\n✓ Retrieved {len(results_q1)} rows")

QUERY 1: Costos y requests diarios por org y servicio en un rango de fechas
Parámetros: org_id=org_0lzjjege, start_date=2025-07-04, end_date=2025-08-30

CQL Equivalent:
  SELECT org_id, usage_date, service, total_cost_usd, total_requests
  FROM org_daily_usage
  WHERE org_id = 'org_0lzjjege' AND usage_date >= '2025-07-04' AND usage_date <= '2025-08-30'
  ORDER BY usage_date DESC;

+--------------+------------+------------+------------+------------+
| Org ID       | Date       | Service    | Cost USD   |   Requests |
+==============+============+============+============+============+
| org_0lzjjege | 2025-08-30 | compute    | $17.47     |          2 |
+--------------+------------+------------+------------+------------+
| org_0lzjjege | 2025-08-30 | networking | $0.16      |          0 |
+--------------+------------+------------+------------+------------+
| org_0lzjjege | 2025-08-30 | storage    | $13.31     |          5 |
+--------------+------------+------------+------------+---------

---
## Query 2: Top-N servicios por costo acumulado (últimos 14 días)

**Requerimiento TP**: *"Top-N servicios por costo acumulado en los últimos 14 días para una organización."*

### CQL Equivalent:
```sql
SELECT org_id, service, total_cost_usd 
FROM org_service_costs 
WHERE org_id = ? AND window_days = 14
ORDER BY total_cost_usd DESC
LIMIT 5;
```

In [4]:
# QUERY 2: Top-N servicios por costo acumulado en los últimos 14 días

ORG_ID = sample_org
WINDOW_DAYS = 14  # Fijo en 14 días según requerimiento
TOP_N = 5

print("=" * 100)
print("QUERY 2: Top-N servicios por costo acumulado en los últimos 14 días")
print("=" * 100)
print(f"Parámetros: org_id={ORG_ID}, window_days={WINDOW_DAYS}, top_n={TOP_N}")
print()
print("CQL Equivalent:")
print(f"  SELECT org_id, service, total_cost_usd")
print(f"  FROM org_service_costs")
print(f"  WHERE org_id = '{ORG_ID}' AND window_days = {WINDOW_DAYS}")
print(f"  ORDER BY total_cost_usd DESC LIMIT {TOP_N};")
print()

# Ejecutar query
collection = db.get_collection("org_service_costs")

# Intentar con 14 días primero
cursor = collection.find(
    filter={"org_id": ORG_ID, "window_days": WINDOW_DAYS},
    sort={"total_cost_usd": -1},
    limit=TOP_N
)

results_q2 = []
for doc in cursor:
    results_q2.append([
        doc.get('org_id', ''),
        f"{doc.get('window_days', 0)} days",
        doc.get('service', ''),
        f"${float(doc.get('total_cost_usd', 0)):,.2f}"
    ])

# Fallback a otras ventanas si no hay data de 14 días
if not results_q2:
    for fallback in [7, 30, 90]:
        cursor = collection.find(
            filter={"org_id": ORG_ID, "window_days": fallback},
            sort={"total_cost_usd": -1},
            limit=TOP_N
        )
        for doc in cursor:
            results_q2.append([
                doc.get('org_id', ''),
                f"{doc.get('window_days', 0)} days",
                doc.get('service', ''),
                f"${float(doc.get('total_cost_usd', 0)):,.2f}"
            ])
        if results_q2:
            print(f"Nota: Usando ventana de {fallback} días (data de 14 días no disponible)")
            break

headers = ["Org ID", "Time Window", "Service", "Total Cost USD"]
print(tabulate(results_q2, headers=headers, tablefmt="grid"))
print(f"\n✓ Retrieved {len(results_q2)} services")

QUERY 2: Top-N servicios por costo acumulado en los últimos 14 días
Parámetros: org_id=org_0lzjjege, window_days=14, top_n=5

CQL Equivalent:
  SELECT org_id, service, total_cost_usd
  FROM org_service_costs
  WHERE org_id = 'org_0lzjjege' AND window_days = 14
  ORDER BY total_cost_usd DESC LIMIT 5;

+--------------+---------------+------------+------------------+
| Org ID       | Time Window   | Service    | Total Cost USD   |
+==============+===============+============+==================+
| org_0lzjjege | 14 days       | storage    | $67.45           |
+--------------+---------------+------------+------------------+
| org_0lzjjege | 14 days       | compute    | $59.86           |
+--------------+---------------+------------+------------------+
| org_0lzjjege | 14 days       | networking | $13.52           |
+--------------+---------------+------------+------------------+

✓ Retrieved 3 services


---
## Query 3: Evolución de tickets críticos y SLA breach (últimos 30 días)

**Requerimiento TP**: *"Evolución de tickets críticos y tasa de SLA breach por día (últimos 30 días)."*

### CQL Equivalent:
```sql
SELECT date, severity, total_tickets, sla_breach_rate, avg_resolution_hours
FROM tickets_critical_daily
WHERE severity = 'critical'
ORDER BY date DESC
LIMIT 30;
```

In [5]:
# QUERY 3: Evolución de tickets críticos y tasa de SLA breach por día (últimos 30 días)

SEVERITY = "critical"
LIMIT_DAYS = 30

print("=" * 100)
print("QUERY 3: Evolución de tickets críticos y tasa de SLA breach (últimos 30 días)")
print("=" * 100)
print(f"Parámetros: severity={SEVERITY}, limit={LIMIT_DAYS} days")
print()
print("CQL Equivalent:")
print(f"  SELECT date, severity, total_tickets, sla_breach_rate, avg_resolution_hours")
print(f"  FROM tickets_critical_daily")
print(f"  WHERE severity = '{SEVERITY}'")
print(f"  ORDER BY date DESC LIMIT {LIMIT_DAYS};")
print()

# Ejecutar query
collection = db.get_collection("tickets_critical_daily")
cursor = collection.find(
    filter={"severity": SEVERITY},
    sort={"date": -1},
    limit=LIMIT_DAYS
)

results_q3 = []
total_tickets = 0
breach_rates = []

for doc in cursor:
    tickets = int(doc.get('total_tickets', 0))
    breach_rate = float(doc.get('sla_breach_rate', 0))
    
    results_q3.append([
        doc.get('date', ''),
        doc.get('severity', ''),
        tickets,
        f"{breach_rate * 100:.1f}%",
        f"{float(doc.get('avg_resolution_hours', 0)):.1f}h"
    ])
    total_tickets += tickets
    breach_rates.append(breach_rate)

headers = ["Date", "Severity", "Total Tickets", "SLA Breach Rate", "Avg Resolution"]
print(tabulate(results_q3, headers=headers, tablefmt="grid"))

if results_q3:
    avg_breach = sum(breach_rates) / len(breach_rates)
    print(f"\n✓ Retrieved {len(results_q3)} days of data")
    print(f"Summary: {total_tickets} total tickets, {avg_breach*100:.1f}% avg SLA breach rate")
else:
    print("\n⚠ No critical tickets found")

QUERY 3: Evolución de tickets críticos y tasa de SLA breach (últimos 30 días)
Parámetros: severity=critical, limit=30 days

CQL Equivalent:
  SELECT date, severity, total_tickets, sla_breach_rate, avg_resolution_hours
  FROM tickets_critical_daily
  WHERE severity = 'critical'
  ORDER BY date DESC LIMIT 30;

+------------+------------+-----------------+-------------------+------------------+
| Date       | Severity   |   Total Tickets | SLA Breach Rate   | Avg Resolution   |
+============+============+=================+===================+==================+
| 2025-08-31 | critical   |               1 | 0.0%              | 168.0h           |
+------------+------------+-----------------+-------------------+------------------+
| 2025-08-28 | critical   |               1 | 0.0%              | 168.0h           |
+------------+------------+-----------------+-------------------+------------------+
| 2025-08-27 | critical   |               1 | 0.0%              | 48.0h            |
+---------

---
## Query 4: Revenue mensual con créditos/impuestos (normalizado a USD)

**Requerimiento TP**: *"Revenue mensual con créditos/impuestos aplicados (normalizado a USD)."*

### CQL Equivalent:
```sql
SELECT org_id, year_month, org_name, total_billed_usd, total_credits_usd, 
       total_taxes_usd, net_revenue, invoice_count
FROM revenue_monthly
WHERE org_id = ?
ORDER BY year_month DESC
LIMIT 12;
```

In [6]:
# QUERY 4: Revenue mensual con créditos/impuestos aplicados (normalizado a USD)

ORG_ID = sample_org
LIMIT_MONTHS = 12

print("=" * 100)
print("QUERY 4: Revenue mensual con créditos/impuestos (normalizado a USD)")
print("=" * 100)
print(f"Parámetros: org_id={ORG_ID}, limit={LIMIT_MONTHS} months")
print()
print("CQL Equivalent:")
print(f"  SELECT org_id, year_month, org_name, total_billed_usd, total_credits_usd,")
print(f"         total_taxes_usd, net_revenue, invoice_count")
print(f"  FROM revenue_monthly")
print(f"  WHERE org_id = '{ORG_ID}'")
print(f"  ORDER BY year_month DESC LIMIT {LIMIT_MONTHS};")
print()

# Ejecutar query
collection = db.get_collection("revenue_monthly")
cursor = collection.find(
    filter={"org_id": ORG_ID},
    sort={"year_month": -1},
    limit=LIMIT_MONTHS
)

results_q4 = []
total_billed = 0
total_credits = 0
total_taxes = 0
total_net = 0

for doc in cursor:
    billed = float(doc.get('total_billed_usd', 0))
    credits = float(doc.get('total_credits_usd', 0))
    taxes = float(doc.get('total_taxes_usd', 0))
    net = float(doc.get('net_revenue', 0))
    
    results_q4.append([
        doc.get('org_id', ''),
        doc.get('year_month', ''),
        (doc.get('org_name', 'N/A') or 'N/A')[:20],
        f"${billed:,.2f}",
        f"${credits:,.2f}",
        f"${taxes:,.2f}",
        f"${net:,.2f}",
        int(doc.get('invoice_count', 0))
    ])
    
    total_billed += billed
    total_credits += credits
    total_taxes += taxes
    total_net += net

headers = ["Org ID", "Month", "Org Name", "Billed USD", "Credits USD", "Taxes USD", "Net Revenue", "Invoices"]
print(tabulate(results_q4, headers=headers, tablefmt="grid"))

if results_q4:
    print(f"\n✓ Retrieved {len(results_q4)} months")
    print(f"Totals: Billed=${total_billed:,.2f}, Credits=${total_credits:,.2f}, Taxes=${total_taxes:,.2f}, Net=${total_net:,.2f}")
else:
    print("\n⚠ No revenue data found for this organization")

QUERY 4: Revenue mensual con créditos/impuestos (normalizado a USD)
Parámetros: org_id=org_0lzjjege, limit=12 months

CQL Equivalent:
  SELECT org_id, year_month, org_name, total_billed_usd, total_credits_usd,
         total_taxes_usd, net_revenue, invoice_count
  FROM revenue_monthly
  WHERE org_id = 'org_0lzjjege'
  ORDER BY year_month DESC LIMIT 12;

+--------------+--------------------------+----------------+--------------+---------------+-------------+---------------+------------+
| Org ID       | Month                    | Org Name       | Billed USD   | Credits USD   | Taxes USD   | Net Revenue   |   Invoices |
+==============+==========================+================+==============+===============+=============+===============+============+
| org_0lzjjege | 2025-08-01T03:00:00.000Z | Alpha Cloud 77 | $2.59        | $0.03         | $0.54       | $3.11         |          1 |
+--------------+--------------------------+----------------+--------------+---------------+-------------

---
## Query 5: Tokens GenAI y costo estimado por día

**Requerimiento TP**: *"Tokens GenAI y costo estimado por día (si existen)."*

**Nota**: Los tokens GenAI solo existen en eventos con schema_version = 2 (después del 2025-07-18).

### CQL Equivalent:
```sql
SELECT org_id, usage_date, total_genai_tokens, total_cost_usd, cost_per_million_tokens
FROM genai_tokens_daily
WHERE org_id = ?
ORDER BY usage_date DESC
LIMIT 30;
```

In [9]:
# QUERY 5: Tokens GenAI y costo estimado por día (si existen)

LIMIT_DAYS = 30

print("=" * 100)
print("QUERY 5: Tokens GenAI y costo estimado por día")
print("=" * 100)
print("Nota: GenAI tokens solo existen en eventos schema v2 (después de 2025-07-18)")
print()

# Obtenemos la colección y buscamos una org que tenga datos GenAI
genai_collection = db.get_collection("genai_tokens_daily")
sample_genai_doc = genai_collection.find_one({})

if sample_genai_doc:
    GENAI_ORG_ID = sample_genai_doc.get('org_id')
else:
    GENAI_ORG_ID = sample_org  # Fallback

print(f"Parámetros: org_id={GENAI_ORG_ID}, limit={LIMIT_DAYS}")
print()
print("CQL Equivalent:")
print(f"  SELECT org_id, usage_date, total_genai_tokens, total_cost_usd, cost_per_million_tokens")
print(f"  FROM genai_tokens_daily")
print(f"  WHERE org_id = '{GENAI_ORG_ID}'")
print(f"  ORDER BY usage_date DESC LIMIT {LIMIT_DAYS};")
print()

# Ejecutar query
cursor = genai_collection.find(
    filter={"org_id": GENAI_ORG_ID},
    sort={"usage_date": -1},
    limit=LIMIT_DAYS
)

results_q5 = []
total_tokens = 0
total_cost = 0

for doc in cursor:
    tokens = int(doc.get('total_genai_tokens', 0))
    cost = float(doc.get('total_cost_usd', 0))
    
    results_q5.append([
        doc.get('org_id', ''),
        doc.get('usage_date', ''),
        f"{tokens:,}",
        f"${cost:,.2f}",
        f"${float(doc.get('cost_per_million_tokens', 0)):,.2f}/M"
    ])
    
    total_tokens += tokens
    total_cost += cost

if results_q5:
    headers = ["Org ID", "Date", "Total Tokens", "Cost USD", "Cost per Million"]
    print(tabulate(results_q5, headers=headers, tablefmt="grid"))
    print(f"\n✓ Retrieved {len(results_q5)} days of GenAI usage")
    print(f"Totals: {total_tokens:,} tokens, ${total_cost:,.2f} cost")
else:
    print("⚠ No GenAI token data found")
    print("  Esto es esperado si los datos solo contienen eventos schema v1 (antes del 2025-07-18)")

QUERY 5: Tokens GenAI y costo estimado por día
Nota: GenAI tokens solo existen en eventos schema v2 (después de 2025-07-18)

Parámetros: org_id=org_n9j2qp89, limit=30

CQL Equivalent:
  SELECT org_id, usage_date, total_genai_tokens, total_cost_usd, cost_per_million_tokens
  FROM genai_tokens_daily
  WHERE org_id = 'org_n9j2qp89'
  ORDER BY usage_date DESC LIMIT 30;

+--------------+------------+----------------+------------+--------------------+
| Org ID       | Date       | Total Tokens   | Cost USD   | Cost per Million   |
+==============+============+================+============+====================+
| org_n9j2qp89 | 2025-08-31 | 3,587          | $16.97     | $4,732.14/M        |
+--------------+------------+----------------+------------+--------------------+
| org_n9j2qp89 | 2025-08-30 | 3,097          | $0.64      | $206.88/M          |
+--------------+------------+----------------+------------+--------------------+
| org_n9j2qp89 | 2025-08-29 | 4,399          | $25.42     | $5,7

---
## Resumen de Ejecución

Este notebook demuestra las **5 consultas mínimas requeridas** del TP, ejecutadas contra **AstraDB** usando la **Data API** (MongoDB-compatible).

### Diseño Query-First
Las tablas en Cassandra/AstraDB fueron diseñadas específicamente para soportar estas consultas:

| Colección | Consulta que soporta |
|-----------|---------------------|
| `org_daily_usage` | Query 1: Costos diarios por org/servicio |
| `org_service_costs` | Query 2: Top-N servicios por costo |
| `tickets_critical_daily` | Query 3: Evolución tickets críticos |
| `revenue_monthly` | Query 4: Revenue mensual normalizado |
| `genai_tokens_daily` | Query 5: Tokens GenAI por día |

In [8]:
# Resumen de resultados
print("=" * 100)
print("RESUMEN DE EJECUCIÓN - 5 CONSULTAS REQUERIDAS")
print("=" * 100)
print()

summary = [
    ["Query 1", "Costos y requests diarios (rango fechas)", len(results_q1), "✓" if results_q1 else "⚠"],
    ["Query 2", "Top-N servicios por costo (14 días)", len(results_q2), "✓" if results_q2 else "⚠"],
    ["Query 3", "Tickets críticos + SLA breach (30 días)", len(results_q3), "✓" if results_q3 else "⚠"],
    ["Query 4", "Revenue mensual (USD normalizado)", len(results_q4), "✓" if results_q4 else "⚠"],
    ["Query 5", "Tokens GenAI y costo por día", len(results_q5), "✓" if results_q5 else "⚠ (esperado si no hay schema v2)"]
]

headers = ["Query", "Descripción", "Rows", "Status"]
print(tabulate(summary, headers=headers, tablefmt="grid"))
print()
print("✓ Todas las consultas ejecutadas exitosamente desde AstraDB")

RESUMEN DE EJECUCIÓN - 5 CONSULTAS REQUERIDAS

+---------+------------------------------------------+--------+----------------------------------+
| Query   | Descripción                              |   Rows | Status                           |
+=========+==========================================+========+==================================+
| Query 1 | Costos y requests diarios (rango fechas) |     20 | ✓                                |
+---------+------------------------------------------+--------+----------------------------------+
| Query 2 | Top-N servicios por costo (14 días)      |      3 | ✓                                |
+---------+------------------------------------------+--------+----------------------------------+
| Query 3 | Tickets críticos + SLA breach (30 días)  |     20 | ✓                                |
+---------+------------------------------------------+--------+----------------------------------+
| Query 4 | Revenue mensual (USD normalizado)        |      3 